## Transformers for Counterfactual Recognition

In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='../input/counterfactualrecognition/subtask1_train_bert.csv')

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=100)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
train_test_dataset = tokenized_datasets["train"].train_test_split(test_size=0.15)

In [5]:
train_test_dataset

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

In [7]:
import numpy as np
from datasets import load_metric

acc_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
p_metric = load_metric("precision")
r_metric = load_metric("recall")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    prec = p_metric.compute(predictions=predictions, references=labels)
    rec = r_metric.compute(predictions=predictions, references=labels)
    return {'accuracy':acc, 'f1':f1, 'precision':prec, 'recall':rec}

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="cr_bert_trainer", evaluation_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_dataset['train'],
    eval_dataset=train_test_dataset['test'],
    compute_metrics=compute_metrics,
)

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
trainer.train()

In [13]:
import pandas as pd

test_df = pd.read_csv('../input/counterfactualrecognition/subtask1_test_bert.csv')

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('/kaggle/working/cr_bert_trainer/checkpoint-4000', num_labels=2)
model.to(device)

In [28]:
inputs = tokenizer(list(test_df['text']), padding="max_length", truncation=True, max_length=100, return_tensors="pt")

In [32]:
test_size = len(test_df)
batch_size = 8
pred_labels = torch.zeros(test_size)

for i in range(0, test_size, batch_size):
    curr_inputs = {k:v[i:min(test_size, i+batch_size)].to(device)  for k, v in inputs.items()}
    outputs = model(**curr_inputs)
    pred_labels[i:min(test_size, i+batch_size)] = outputs['logits'].argmax(1)

In [33]:
pred_labels = np.array(pred_labels.cpu().numpy())
gold_labels = np.array(test_df['label'])

In [34]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print('F1: ', f1_score(gold_labels, pred_labels))
print('Accuracy: ', accuracy_score(gold_labels, pred_labels))
print('Precision: ', precision_score(gold_labels, pred_labels))
print('Recall: ', recall_score(gold_labels, pred_labels))